In [2]:
pip install --quiet -U langchain langchain-chroma langchain-ollama


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'dodo'

llama_model = 'llama3.2:3b-instruct-fp16'

In [ ]:

os.getenv('LANGCHAIN_API_KEY')

In [ ]:
from langchain_core.documents import Document

documents = [
  Document(
      page_content="Dogs are great companions, known for their loyalty and friendliness.",
      metadata={"source": "mammal-pets-doc"},
  ),
  Document(
      page_content="Cats are independent pets that often enjoy their own space.",
      metadata={"source": "mammal-pets-doc"},
  ),
  Document(
      page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
      metadata={"source": "fish-pets-doc"},
  ),
  Document(
      page_content="Parrots are intelligent birds capable of mimicking human speech.",
      metadata={"source": "bird-pets-doc"},
  ),
  Document(
      page_content="Rabbits are social animals that need plenty of space to hop around.",
      metadata={"source": "mammal-pets-doc"},
  ),
]


In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

vectorstore = Chroma.from_documents(
  documents,
  embedding=OllamaEmbeddings(model=llama_model)
)

In [ ]:
vectorstore.similarity_search("cat")

In [ ]:
await vectorstore.asimilarity_search("cat")

In [ ]:
vectorstore.similarity_search_with_score("cat")

In [ ]:
embedding = OllamaEmbeddings(model=llama_model).embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) # top result

retriever.batch(["cat", "shark"])

In [ ]:
retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={"k":1}
)

retriever.batch(["cat", "shark"])

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model=llama_model)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [ ]:
response = rag_chain.invoke("tell me about cats")

print(response.content)